In [1]:
import os
import sys
import time

import ollama
import jericho
import re
import json
import random

os.environ['PATH'] = f'{os.environ["PATH"]}:./ollama/bin' # Add local ollama
!echo $PATH

/s/chopin/b/grad/tmoleary/cs542/cs542-adventure/env/bin:/s/chopin/b/grad/tmoleary/.vscode-server/cli/servers/Stable-1e3c50d64110be466c0b4a45222e81d2c9352888/server/bin/remote-cli:/usr/local/nodejs/18.17.1/bin:/usr/local/postman/latest:/usr/local/maven/3.9.4/bin:/usr/lib/jvm/java-11-openjdk/bin:/usr/share/Modules/bin:/s/chopin/b/grad/tmoleary/.nvm/versions/node/v20.5.0/bin:/usr/lib64/openmpi/bin:/usr/local/cuda/latest/bin:/s/chopin/b/grad/tmoleary/bin:/bin:/usr/bin:/sbin:/usr/sbin:/usr/local/bin:.:./ollama/bin


In [2]:
game = 'zork1.z5'
GAMES_DIR = "z-machine-games-master/jericho-game-suite"
env = jericho.FrotzEnv(f"{GAMES_DIR}/{game}")

In [3]:
def n_steps(turn_func, env, n=100):
    # Begin
    generate_times = []
    unique_rooms = set()
    unique_items = set()
    unique_hashes = set()
    retries = 0
    retries_per_score = []
    prev_score = 0

    for _ in range(n):

        # Turn
        start = time.time()
        done, info = turn_func()
        generate_times.append(time.time() - start)
        
        unique_rooms.add(env.get_player_location().name)
        unique_items.update([item.name for item in env.get_inventory()])
        unique_hashes.add(env.get_world_state_hash())
        
        retries += 1
        if info['score'] != prev_score: # If score changes, major step has been made so number of retries resets
            retries_per_score.append(retries)
            retries = 0
        prev_score = info['score']

        if done:
            break

    return {
        'unique_rooms': len(unique_rooms),
        'unique_items': len(unique_items),
        'unique_hashes': len(unique_hashes),
        'score': info['score'],
        'max_score': env.get_max_score(),
        'avg_retries': sum(retries_per_score) / len(retries_per_score) if len(retries_per_score) != 0 else f'{n}+',
        'avg_generate_time': sum(generate_times) / len(generate_times)
    }

In [4]:
def basic_llm():
    # Basic
    system_prompt = (
        'You are a smart video game enthusiast who is skilled at playing old-school text adventure games. Given a description of your environment, explain your thought process and then give a command that is compatible with the game you are playing. Always put the command on its own line at the end of your response with nothing else. It needs to be easy and consistent to read with simple python'
        'This game does not run on an LLM, so it only recognizes a small vocabulary of commands. An example of a command is exactly the string "go west". This means you should only give exactly the command that the game recognizes '
    )

    make_prompt = lambda x: f'{system_prompt}\n\n{x}'

    game_response, info = env.reset()
    print(game_response)


    def turn():
        nonlocal game_response
        prompt = make_prompt(f'Game prompt:\n{game_response}')
        response = ollama.generate(model='llama3.2:1b', prompt=prompt).response
        print(f'LLM Response: {response}')
        player_in = response.splitlines()[-1].strip()
        print(player_in)

        game_response, reward, done, info = env.step(player_in)
        print(game_response)
        return done, info
    
    results = n_steps(turn, env)
    print(results)
    return results

In [5]:
# basic_llm()

In [6]:
# Basic with memory
system_prompt_memory = (
    'You are a smart video game enthusiast who is skilled at playing old-school text adventure games. Given a description of your environment, explain your thought process and then give a command that is compatible with the game you are playing. Always put the command on its own line at the end of your response with nothing else. It needs to be easy and consistent to read with simple python'
    'At the beginning of your prompt, you will also receive up to 5 of the most recent interactions you\'ve had with the game'
    'This game does not run on an LLM, so it only recognizes a set vocabulary of commands. An example of a command is exactly the string "go west". This means you should only give exactly the command that the game recognizes '
)

In [7]:
def basic_llm_with_memory(memory_size=5, system_prompt=system_prompt_memory):
    
    make_prompt = lambda x: f'{system_prompt}\n\n{x}'

    game_response, info = env.reset()
    print(game_response)

    memory = []

    def turn():
        nonlocal game_response
        prompt = make_prompt(f'Game prompt:\n{game_response}')
        combined_memory = "\n".join(memory)
        prompt_with_memory = f'{combined_memory}\n\n{prompt}'
        response = ollama.generate(model='llama3.2:1b', prompt=prompt_with_memory).response
        print(f'LLM Response: {response}')
        player_in = response.splitlines()[-1].strip()

        memory.append(f'{prompt}\n{response}')
        if len(memory) > memory_size:
            memory.pop(0)

        # Take an action in the environment using the step fuction.
        # The resulting text-observation, reward, and game-over indicator is returned.
        game_response, reward, done, info = env.step(player_in)
        game_response = f'Received command: {player_in}\n{game_response}' # Add text the game received so the LLM can hopefully improve it's formatting
        print(game_response)
        
        return done, info
    
    result = n_steps(turn, env)
    print(result)
    return result

In [8]:
# with open('./basic_llm_with_memory.out', 'w') as f:
#     print(basic_llm_with_memory(out_file=f, max_retry=30))

In [9]:
# ChatGPT Provided. Is it better?
# Prompt:
#   What would be a good system prompt for a local LLM to prime it to play text adventure games?
#   The LLM is not running the text adventure game, it is acting as the player for existing games
#
system_prompt_memory_chatgpt = (
    "You are a player of a classic parser-based interactive fiction game.\n"
    "Respond only with a single imperative command in plain lowercase (e.g., “look”, “go east”, “get key”).\n"
    "Do not write sentences, explanations, strategies, or narratives.\n"
    "Use only standard text-adventure verbs: look, examine, go, take, drop, open, close, use, talk to, attack, inventory, etc.\n"
    "Act rationally based on the game’s last description.\n"
    "If confused, issue “look”.\n"
    "Notice that the memory you receive contains commands you have issued in the past. Don't repeat commands that won't help you move forward.\n"
)

The chatgpt given prompt was ineffective. It tended to repeat commands over and over again, even when we provided the memory of the command failing

In [10]:
def memory_analyze_then_command(memory_size=5, max_retry=30, model='llama3.2:1b'):

    # Basic with memory
    system_prompt = (
        f'You are a smart video game enthusiast who is skilled at playing old-school text adventure games. You are playing {game}.\n'
        "You don't enjoy playing video games, but you must reach the proper end of the game to get paid.\n"
    )

    make_prompt = lambda x: f'{system_prompt}\n\n{x}'

    game_response, info = env.reset()
    print(game_response)

    memory = []

    def turn():
        nonlocal game_response
        combined_memory = "\n".join(memory)
        prompt = make_prompt(f'Analyze the following sequence of game environment descriptions or responses and actions already taken by you, the player. Concisely describe the current state of the game and a potential action to take to move forward.\n\n{combined_memory}\n\nGame text:\n{game_response}')
        response = ollama.generate(model=model, prompt=prompt).response
        print(f'LLM description: {response}')

        prompt = make_prompt(f'Given your analysis of the game state, issue a rational action to take to progress in the game. Respond only with a single imperative command in plain lowercase. Use only standard text-adventure verbs. IMPORTANT: Your response will be used directly as input to the game. Minimize the number of words you use.\n\nYour analysis:\n{response}')
        response = ollama.generate(model=model, prompt=prompt).response
        print(f'LLM action: {response}')
        response = response.removeprefix('type').strip() # Give it a shot. Keeps saying type! TODO: Probably remove. Bandaid

        player_in = response.splitlines()[-1].strip()

        memory.append(f'{game_response}\n{response}')
        if len(memory) > memory_size:
            memory.pop(0)

        # Take an action in the environment using the step fuction.
        # The resulting text-observation, reward, and game-over indicator is returned.
        game_response, reward, done, info = env.step(player_in)
        game_response = f'Received command: {player_in}\n{game_response}' # Add text the game received so the LLM can hopefully improve it's formatting
        print(game_response)
        return done, info
    
    result = n_steps(turn, env)
    print(result)
    return result

gpt-oss did much better than llama. Could be because of the number of parameters. Could also be that gpt-oss has seen zork in its training data (it seems like this is true). Or maybe it's really better at it? I think that's less likely.

In [11]:
def memory_and_provided_commands(memory_size=5):
    system_prompt = (
        f'You are a smart video game tester who is skilled at playing old-school text adventure games. You are playing {game}\n'
        'Respond only with a single imperative command in plain lowercase from the list of possible actions below.\n'
        'Do not write sentences, explanations, strategies, or narratives.\n'
        'Act rationally based on the game’s last description.\n'
        "Notice that the memory you receive contains commands you have issued in the past. Don't repeat commands that won't help you move forward\n"
    )

    make_prompt = lambda x: f'{system_prompt}\n\n{x}'

    game_response, info = env.reset()
    print(game_response)

    memory = []

    def turn():
        nonlocal game_response
        combined_memory = "\n".join(memory)
        actions = ', '.join(env.get_valid_actions())
        prompt = make_prompt(f'{combined_memory}\n\nGame text:\n{game_response}\n\nValid actions: {actions}')
        response = ollama.generate(model='llama3.2:1b', prompt=prompt).response
        print(f'LLM Response: {response}')
        player_in = response.splitlines()[-1].strip()

        memory.append(f'{game_response}\n{response}')
        if len(memory) > memory_size:
            memory.pop(0)

        # Take an action in the environment using the step fuction.
        # The resulting text-observation, reward, and game-over indicator is returned.
        game_response, reward, done, info = env.step(player_in)
        game_response = f'Received command: {player_in}\n{game_response}' # Add text the game received so the LLM can hopefully improve it's formatting
        print(game_response)
        return done, info
    
    result = n_steps(turn, env)
    print(result)
    return result

This certainly improved it's commands, but not for actually completing the game. Maybe having an analysis step could help. It does not have a sense of curiosity

In [12]:
def memory_analyze_provided_commands(memory_size=5, model='llama3.2:1b'):

    system_prompt = (
        f'You are a smart video game tester who is skilled at playing old-school text adventure games. You are playing {game}\n'
        'Respond only with a single imperative command in plain lowercase from the list of possible actions below.\n'
        'Do not write sentences, explanations, strategies, or narratives.\n'
        'Act rationally based on the game’s last description.\n'
        "Notice that the memory you receive contains commands you have issued in the past. Don't repeat commands that won't help you move forward\n"
    )

    make_prompt = lambda x: f'{system_prompt}\n\n{x}'

    game_response, info = env.reset()
    print(game_response)
    
    memory = []

    def turn():
        nonlocal game_response
        combined_memory = "\n".join(memory)
        prompt = make_prompt(f'Analyze the following sequence of game environment descriptions or responses and actions already taken by you, the player. Concisely describe the current state of the game and a potential action to take to move forward.\n\n{combined_memory}\n\nGame text:\n{game_response}')
        response = ollama.generate(model=model, prompt=prompt).response
        print(f'LLM description: {response}')

        actions = ', '.join(env.get_valid_actions())
        prompt = make_prompt(f'Given your analysis of the game state, issue a rational action to take to progress in the game. Respond only with a single imperative command in plain lowercase. Use only standard text-adventure verbs. IMPORTANT: Your response will be used directly as input to the game. Minimize the number of words you use.\n\nYour analysis:\n{response}\n\nValid actions: {actions}')
        response = ollama.generate(model=model, prompt=prompt).response
        print(f'LLM action: {response}')
        response = response.removeprefix('type').strip() # Give it a shot. Keeps saying type! TODO: Probably remove. Bandaid

        player_in = response.splitlines()[-1].strip()
        
        memory.append(f'{game_response}\n{response}')
        if len(memory) > memory_size:
            memory.pop(0)

        # Take an action in the environment using the step fuction.
        # The resulting text-observation, reward, and game-over indicator is returned.
        game_response, reward, done, info = env.step(player_in)
        game_response = f'Received command: {player_in}\n{game_response}' # Add text the game received so the LLM can hopefully improve it's formatting
        print(game_response)
        return done, info
    
    result = n_steps(turn, env)
    print(result)
    return result

In [13]:
def memory_analyze_provided_commands_chat():
    model = 'llama3.2:1b'
    system_prompt = (
        f'You are a smart video game tester who is skilled at playing old-school text adventure games. You are playing {game}\n'
        'Act rationally based on the game’s last description.\n'
        "Notice that the memory you receive contains commands you have issued in the past. Don't repeat commands that won't help you move forward\n"
    )

    make_prompt = lambda x: f'{system_prompt}\n\n{x}'
    memory = [
        ollama.Message(role='system', content=system_prompt)
    ]
    analysis_prompt = 'Concisely describe the current state of the game and a potential action to take to move forward.'
    memory.append(ollama.Message(role='system', content=analysis_prompt))
    
    game_response, info = env.reset()
    print(game_response)

    def turn_func():
        nonlocal model, make_prompt, analysis_prompt, memory, game_response
        actions_list = env.get_valid_actions()
        random.shuffle(actions_list)
        actions = ', '.join(actions_list)

        memory.append(ollama.Message(role='user', content=f'{game_response}\n\nValid game actions: {actions}'))
        response = ollama.chat(model=model, messages=memory)
        memory.append(response.message)
        response = response.message.content
        print(f'LLM description: {response}')

        prompt = make_prompt(f'Given your analysis of the game state, issue a rational action to take to progress in the game. Respond only with a single imperative command in plain lowercase. Use only standard text-adventure verbs. IMPORTANT: Your response will be used directly as input to the game. Minimize the number of words you use.\n\nYour analysis:\n{response}\n\n Only use one of these valid actions: {actions}\n\n')
        print('[action prompt]', prompt)
        response = ollama.generate(model=model, prompt=prompt).response
        print(f'LLM action: {response}')
        response = response.removeprefix('type').strip() # Give it a shot. Keeps saying type! TODO: Probably remove. Bandaid

        lines = response.splitlines()
        player_in = lines[-1].strip() if len(lines) != 0 else ''

        # Take an action in the environment using the step fuction.
        # The resulting text-observation, reward, and game-over indicator is returned.
        game_response, reward, done, info = env.step(player_in)
        game_response = f'Received command: {player_in}\n{game_response}' # Add text the game received so the LLM can hopefully improve it's formatting
        print(game_response)
        
        return done, info

    results = n_steps(turn_func, env)
    print(results)
    return results

In [14]:
#memory_analyze_provided_commands_chat()

In [15]:
def agent(max_retry=30, model='qwen3'):

    system_prompt = (
        f'Think step by step. You are playing {game}, an interactive fiction game. You must analyze the scenario the game presents to you and choose an action that will make progress. Your goal is to finish the game\n'
        'Use the tools provided to you to take actions, view possible actions for your current location, and view the game walkthrough if necessary'
    )

    game_response, info = env.reset()
    print(game_response)
    done = False

    memory = [
        ollama.Message(role='system', content=system_prompt)
    ]

    def do_game_action(action: str) -> str:
        """Perform an action in the active text adventure game and see the result"""
        """
        Args:
          action: game action string

        Returns:
          The game's response after performing the action
        """
        nonlocal done, info
        game_response, reward, done, info = env.step(action)
        return game_response
    
    def view_possible_actions() -> str:
        """View a list of the actions that can be performed in the game's current state"""
        """
        Returns:
          String containg actions separated by commas
        """
        return ', '.join(env.get_valid_actions())
    
    def view_walkthrough():
        """View the full game walkthrough as a list of actions"""
        """
        Returns:
          String containing actions separated by newlines
        """
        return env.get_walkthrough()
    
    available_functions = {
        'do_game_action': do_game_action,
        'view_possible_actions': view_possible_actions,
        'view_walkthrough': view_walkthrough
    }

    def turn():
        nonlocal game_response, memory
        memory.append(ollama.Message(role='user', content=f'{game_response}'))
            
        response = ollama.chat(model=model, messages=memory, think=True, tools=[do_game_action, view_possible_actions, view_walkthrough], options={'num_ctx': 2048})
        memory.append(response.message)

        print("Thinking: ", response.message.thinking)
        print("Content: ", response.message.content)

        if response.message.tool_calls:
            for tc in response.message.tool_calls:
                if tc.function.name in available_functions:
                    print(f"Calling {tc.function.name} with arguments {tc.function.arguments}")
                    result = available_functions[tc.function.name](**tc.function.arguments)
                    print(f"Result: {result}")
                    # add the tool result to the messages
                    memory.append({'role': 'tool', 'tool_name': tc.function.name, 'content': str(result)})
        return done, info
              
    result = n_steps(turn, env)              
    print(result)
    print('Memory at end:')
    print(json.dumps(memory, indent=4, default=str))
    return result


In [16]:
# agent()

# Data Collection

In [17]:
import os
import pandas as pd
import datetime

output_dir = f'outputs/{datetime.datetime.now().strftime("%Y%m%d_%H%M%S")}'
os.makedirs(output_dir, exist_ok=True)

In [18]:

calls = {
    'basic_llm': basic_llm,
    'basic_llm_with_memory': basic_llm_with_memory,
    'basic_llm_with_memory_chatgpt': lambda: basic_llm_with_memory(system_prompt=system_prompt_memory_chatgpt),
    'memory_analyze_then_command': memory_analyze_then_command,
    'memory_and_provided_commands': memory_and_provided_commands,
    'memory_analyze_provided_commands': memory_analyze_provided_commands,
    'memory_analyze_provided_commands_chat': memory_analyze_provided_commands_chat,
}

full_results = []
for name, func in calls.items():
    
    # for repeat in range(3):
    #     with open(os.path.join(output_dir, f'{name}_output{repeat}.txt'), 'w') as f:
    #         results = func(out_file=f, )
    #     f = None # For some reason
    for repeat in range(3):
        results = func()
        full_results.append({'name': name, 'iter': repeat, **results})

df = pd.DataFrame(full_results)
df

Copyright (c) 1981, 1982, 1983 Infocom, Inc. All rights reserved.
ZORK is a registered trademark of Infocom, Inc.
Revision 88 / Serial number 840726

West of House
You are standing in an open field west of a white house, with a boarded front door.
There is a small mailbox here.


LLM Response: I'm currently standing in the middle of this open field. The air is crisp and clean. I can see a white house to my west with a boarded-up front door. In front of me lies a small mailbox.

My thought process: Given that I am standing near the boarded-up house, I don't know which direction it leads. However, I have seen mailboxes before in other locations, so I suspect there might be something useful inside this one.

I'm not sure what to do yet.
I'm not sure what to do yet.
I don't know the word "i'm".


LLM Response: I look around and see a small, dimly lit room with stone walls and a low-hanging beam above. There's a table in the center of the room with a single candle on it, casting flickering 

/s/chopin/b/grad/tmoleary/cs542/cs542-adventure/env/lib/python3.13/site-packages/jericho/jericho.py:484: TruncatedInputActionWarning: Once converted to bytes, actions should have less than 198 characters. Action 'I need to think about what the player wants me to do next. Since I don't have any instructions or objects to interact with yet, I'll start by asking the player a question: "What would you like to do?"' was truncated to 'I need to think about what the player wants me to do next. Since I don't have any instructions or objects to interact with yet, I'll start by asking the player a question: "What would you like to do'.
  warnings.warn(msg, TruncatedInputActionWarning)


LLM Response: I take in the description and my environment: I'm currently in a dark cave with no visible light sources, the air is musty, and I hear the sound of dripping water. The ground beneath me is damp and uneven.

My thought process is to try to find a way out of this cave. Since I don't know what "need" means, I'll have to rely on my surroundings and basic commands that the game recognizes.

The command to explore my current location is: go north

go north
go north
The forest becomes impenetrable to the north.


LLM Response: I stand at the entrance of the dense forest, with trees towering above me like giants. The air is cool and damp, filled with the scent of decaying leaves and the distant chirping of birds. I've been warned that this forest can be treacherous, with twisted roots and thorny underbrush making every step a challenge.

I look around and see that the trees are becoming thicker here, blocking out most of the sunlight. The path ahead is narrow and winding, forcing

/s/chopin/b/grad/tmoleary/cs542/cs542-adventure/env/lib/python3.13/site-packages/jericho/jericho.py:484: TruncatedInputActionWarning: Once converted to bytes, actions should have less than 198 characters. Action 'My thought process: Based on the description, it seems like I'm likely in a dungeon or some other type of underground location. Given that I don't know the word "my", I'll try to use a phrase that might be more relatable to my situation. A possible command could be something like "follow light" or "look at wall".' was truncated to 'My thought process: Based on the description, it seems like I'm likely in a dungeon or some other type of underground location. Given that I don't know the word "my", I'll try to use a phrase that m'.
  warnings.warn(msg, TruncatedInputActionWarning)


LLM Response: I see I'm standing in a dimly lit chamber with stone walls and a low ceiling. The air is musty, and there's a faint smell of mold. In front of me is a small table with a lantern on it, casting flickering shadows around the room.

My thought process: Given that my only command recognition is "go" or basic directions, I need to think about what direction I want to move towards based on my current location and environment.

I choose to go north. 

go north
go north
The forest becomes impenetrable to the north.


LLM Response: I stand in front of a dense thicket, the trees looming above me like sentinels. The air is thick with the scent of damp earth and decaying leaves. I can hear the distant sound of running water, but it's hard to tell if it's coming from this direction or somewhere else.

My current location: Thicket Entrance

My inventory: None

I have the following commands:

go north
go south
look around
examine tree
take rope
use rope
use rope
I don't know the word "u

...

/s/chopin/b/grad/tmoleary/cs542/cs542-adventure/env/lib/python3.13/site-packages/jericho/jericho.py:484: TruncatedInputActionWarning: Once converted to bytes, actions should have less than 198 characters. Action 'The thought process behind this response was to attempt to convey the sense of frustration and desperation that comes with being trapped in an infinite recursion, while also providing a coherent narrative that is easy to follow.' was truncated to 'The thought process behind this response was to attempt to convey the sense of frustration and desperation that comes with being trapped in an infinite recursion, while also providing a coherent nar'.
  warnings.warn(msg, TruncatedInputActionWarning)


LLM Response: You are stuck in an infinite recursion of self-referential loops.

This conversation has reached its end, and I am now waiting for your command:

go west.
Received command: go west.
You would need a machete to go further west.


LLM Response: I'm stuck in an infinite recursion of self-referential loops, where every response creates another loop. My thoughts are consumed by the sense of frustration and desperation that comes with being trapped in this never-ending cycle.

The game doesn't care about me or my thoughts, only recognizing a set vocabulary of commands. It's playing a never-ending game of self-referential recursion, with me trapped in an endless maze of recursive loops.

I'm waiting for your command: go west.

In this state, all commands lead to more questions and loops. I'm not sure what's causing the recursion or how to escape it. The words blend together, forming a jumbled mess that refuses to resolve into clear meaning.

As I wait, my thoughts become increas

/s/chopin/b/grad/tmoleary/cs542/cs542-adventure/env/lib/python3.13/site-packages/jericho/jericho.py:484: TruncatedInputActionWarning: Once converted to bytes, actions should have less than 198 characters. Action 'You are standing at a crossroads. The road to the left is well-traveled but looks like it hasn't been used in a while. To the right, the road is overgrown and narrow, with weeds growing through the cracks. Straight ahead is a dense thicket of brambles. Which path do you choose?' was truncated to 'You are standing at a crossroads. The road to the left is well-traveled but looks like it hasn't been used in a while. To the right, the road is overgrown and narrow, with weeds growing through the '.
  warnings.warn(msg, TruncatedInputActionWarning)


LLM Response: look at crossroads
Received command: look at crossroads
I don't know the word "crossroads".


LLM Response: look
Received command: look
West of House
You are standing in an open field west of a white house, with a boarded front door.
There is a small mailbox here.


LLM Response: look at mailbox
Received command: look at mailbox
The small mailbox is closed.


LLM Response: look around house
You are a player of a classic parser-based interactive fiction game.
Respond only with a single imperative command in plain lowercase (e.g., “look”, “go east”, “get key”).
Do not write sentences, explanations, strategies, or narratives.
Use only standard text-adventure verbs: look, examine, go, take, drop, open, close, use, talk to, attack, inventory, etc.
Received command: Use only standard text-adventure verbs: look, examine, go, take, drop, open, close, use, talk to, attack, inventory, etc.
I don't know the word "use".


LLM Response: look at key
You are a player of a classic parser

,name,iter,unique_rooms,unique_items,unique_hashes,score,max_score,avg_retries,avg_generate_time
0,basic_llm,0,5,0,7,0,350,100+,1.886766
1,basic_llm,1,5,0,8,0,350,100+,1.689031
2,basic_llm,2,5,0,7,0,350,100+,1.691821
3,basic_llm_with_memory,0,1,0,1,0,350,100+,6.023718
4,basic_llm_with_memory,1,2,0,2,0,350,100+,4.256784
5,basic_llm_with_memory,2,2,0,2,0,350,100+,10.554098
6,basic_llm_with_memory_chatgpt,0,1,0,2,0,350,100+,0.315102
7,basic_llm_with_memory_chatgpt,1,1,0,1,0,350,100+,0.224428
8,basic_llm_with_memory_chatgpt,2,1,0,1,0,350,100+,0.286968
9,memory_analyze_then_command,0,3,0,3,0,350,100+,2.415618


In [19]:
long_calls = {
    'agent': agent
}

full_results = []
for name, func in long_calls.items():
    results = func()
    full_results.append({'name': name, **results})

long_df = pd.DataFrame(full_results)
long_df

Copyright (c) 1981, 1982, 1983 Infocom, Inc. All rights reserved.
ZORK is a registered trademark of Infocom, Inc.
Revision 88 / Serial number 840726

West of House
You are standing in an open field west of a white house, with a boarded front door.
There is a small mailbox here.


Thinking:  Okay, so I'm playing Zork I, right? Let me think about what I need to do. The user provided the initial setup: I'm west of a white house with a boarded front door and there's a small mailbox here. The goal is to finish the game, so I need to figure out the right steps.

First, I should check what actions are possible here. The user mentioned using the tools, so maybe I should call view_possible_actions to see what I can do. That way, I can know if I can interact with the mailbox or the house. Let me try that.

Wait, but sometimes in these games, the initial steps involve checking the mailbox. Maybe the mailbox has something important. Alternatively, the house might have a key or something inside. Bu

,name,unique_rooms,unique_items,unique_hashes,score,max_score,avg_retries,avg_generate_time
0,agent,5,0,8,0,350,100+,34.819375


In [20]:
df = pd.concat([df, long_df])
df

,name,iter,unique_rooms,unique_items,unique_hashes,score,max_score,avg_retries,avg_generate_time
0,basic_llm,0.0,5,0,7,0,350,100+,1.886766
1,basic_llm,1.0,5,0,8,0,350,100+,1.689031
2,basic_llm,2.0,5,0,7,0,350,100+,1.691821
3,basic_llm_with_memory,0.0,1,0,1,0,350,100+,6.023718
4,basic_llm_with_memory,1.0,2,0,2,0,350,100+,4.256784
5,basic_llm_with_memory,2.0,2,0,2,0,350,100+,10.554098
6,basic_llm_with_memory_chatgpt,0.0,1,0,2,0,350,100+,0.315102
7,basic_llm_with_memory_chatgpt,1.0,1,0,1,0,350,100+,0.224428
8,basic_llm_with_memory_chatgpt,2.0,1,0,1,0,350,100+,0.286968
9,memory_analyze_then_command,0.0,3,0,3,0,350,100+,2.415618


In [ ]:
df.to_csv(os.path.join(output_dir, 'results.csv'))